In [1]:
import json

In [2]:
with open("./rosmsgs.json", "r") as jsonFile:
    messages_data = json.load(jsonFile)
messages = sorted(messages_data.keys())

In [3]:
from msginfo import *

In [4]:
def collect_modules(messages):
    modules = {}
    for msg in messages:
        mi = MsgInfo(msg, messages_data[msg])
        if mi.module not in modules:
            modules[mi.module] = {}
        msgdata = \
            { 'info'    : mi
            , 'depends' : mi.get_local_dependencies()
            }    
        modules[mi.module][mi.name] = msgdata  
    return modules

modules = collect_modules(messages)

In [5]:
# Deduplicating identifiers  
for mod_name in modules:
    module = modules[mod_name]
    # Gather duplicate field names
    fields = {}
    for msg in module:
        for f in module[msg]['info'].get_field_names():
            if f not in fields:
                fields[f] = []
            fields[f].append(msg)

    duplicates = {}
    for field, msgs in fields.iteritems():
        if len(msgs) < 2: 
            continue
        for msg in msgs:
            if msg not in duplicates:
                duplicates[msg] = []
            duplicates[msg].append(field)

    # Apply prefix for duplicates
    for msg, fields in duplicates.iteritems():
        module[msg]['info'].prefix_fields(fields)

In [6]:
# Special treatment for std_msgs module
module = modules["Std_msgs"]

def prefix_if_basetype(name):
    basetypes = {
        "time" , "duration" , "byte" , "bool", "char", "string",
        "int8" , "int16" , "int32" , "int64" , 
        "uint8" , "uint16" , "uint32" , "uint64" , 
        "float32" , "float64"  }
    if name in basetypes:
        return "std_" + name
    return name 

for msg in module.keys():
    data = module[msg]
    new_name = prefix_if_basetype(msg)
    data['info'].name = new_name
    for e in data['info'].entries:
        if e['type'].get('module', None) == "Std_msgs":
            e['type']['name'] = prefix_if_basetype(e['type']['name'])
    del module[msg]
    data['depends'] = [prefix_if_basetype(d) for d in data['depends']] 
    module[new_name] = data
module

{u'byteMultiArray': {'depends': [u'multiArrayLayout'],
  'info': <msginfo.MsgInfo at 0x7f81d166b2d0>},
 u'colorRGBA': {'depends': [], 'info': <msginfo.MsgInfo at 0x7f81d166b390>},
 u'empty': {'depends': [], 'info': <msginfo.MsgInfo at 0x7f81d166b410>},
 u'float32MultiArray': {'depends': [u'multiArrayLayout'],
  'info': <msginfo.MsgInfo at 0x7f81d166b490>},
 u'float64MultiArray': {'depends': [u'multiArrayLayout'],
  'info': <msginfo.MsgInfo at 0x7f81d166b550>},
 u'header': {'depends': [], 'info': <msginfo.MsgInfo at 0x7f81d166b590>},
 u'int16MultiArray': {'depends': [u'multiArrayLayout'],
  'info': <msginfo.MsgInfo at 0x7f81d166b690>},
 u'int32MultiArray': {'depends': [u'multiArrayLayout'],
  'info': <msginfo.MsgInfo at 0x7f81d166b710>},
 u'int64MultiArray': {'depends': [u'multiArrayLayout'],
  'info': <msginfo.MsgInfo at 0x7f81defbabd0>},
 u'int8MultiArray': {'depends': [u'multiArrayLayout'],
  'info': <msginfo.MsgInfo at 0x7f81d166b790>},
 u'multiArrayDimension': {'depends': [],
  'in

In [7]:
data['info'].entries

[{'local': True,
  'name': u'int64MultiArray_layout',
  'type': {'module': u'Std_msgs', 'name': u'multiArrayLayout'}},
 {'name': u'int64MultiArray_data', 'type': {'list': True, 'name': u'int64'}}]

In [8]:
def write_types(mod_name, code):
    folder = "../imandra_model/src-messages"
    filename = folder + "/" + mod_name.lower() + ".ml"
    with open(filename, "w") as mlfile:
        mlfile.write("open Basic_types;;\n\n")
        for c in code:
            mlfile.write(c)
            mlfile.write("\n\n")
            
def write_readers(mod_name, code):
    folder = "../imandra_model/src-messages-pp"
    filename = folder + "/" + mod_name.lower() + "_of_json.ml"
    with open(filename, "w") as mlfile:
        mlfile.write("open Json_utils;;\n")
        mlfile.write("open Basic_types_of_json;;\n")        
        mlfile.write("open Ros_messages.{};;\n\n".format(mod_name))        
        for c in code:
            mlfile.write(c)
            mlfile.write("\n\n")   

def write_writers(mod_name, code):
    folder = "../imandra_model/src-messages-pp"
    filename = folder + "/" + mod_name.lower() + "_to_json.ml"
    with open(filename, "w") as mlfile:
        mlfile.write("open Json_utils;;\n")
        mlfile.write("open Basic_types_to_json;;\n")        
        mlfile.write("open Ros_messages.{};;\n\n".format(mod_name))        
        for c in code:
            mlfile.write(c)
            mlfile.write("\n\n")   

In [9]:
def get_type_writer(tinfo, current_module=None):
    tdecl = kw(tinfo['name']) + "_to_json"
    if 'module' in tinfo and tinfo['module'] != current_module:
        tdecl = "{0}_to_json.{1}".format( tinfo['module'] , tdecl )
    if tinfo.get('list', False):
        tdecl = "(mklist {})".format(tdecl)
    return tdecl    
    
def gen_json_writer(self):
        if self.entries == []:
            return "let {0}_to_json x = `Assoc []".format(self.name)
        code = ["let {}_to_json x = [".format(self.name)]
        for entry in self.entries:
            etype = get_type_writer(entry['type'], self.module)
            code.append("    ( \"{0}\" , x.{0} |> {1} );".format(entry['name'], etype))
        code += ["    ] |> assoc_filter_nulls"]
        return "\n".join(code)

In [10]:
for mod_name in modules:
    module = modules[mod_name].copy()
    generated = set()
    types_code, readers_code, writers_code = [], [], []
    while module:
        for m in module:
            deps = module[m]['depends']
            deps = set(deps) - generated 
            if not deps:
                generated.add(m)
                types_code.append(module[m]['info'].gen_typedecl())
                readers_code.append(module[m]['info'].gen_json_reader())
                writers_code.append(gen_json_writer(module[m]['info']))
        for m in generated:
            if m in module:
                del module[m]
    
    write_types(mod_name, types_code)
    write_readers(mod_name, readers_code)
    write_writers(mod_name, writers_code)